In [38]:
import os, sys
import numpy as np
import psycopg2
from subprocess import call
import subprocess

In [5]:
# call(["convert", "-size", "100x100", "xc:wheat", "canvas_wheat.gif"])
# call(["convert", "-size", "100x100", "xc:red", "canvas_red.gif"])

0

In [17]:
# set context/cwd

stack_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/"
channel_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/"

os.chdir(base_path)
cwd = os.getcwd()
print "Current working directory %s" % cwd

Current working directory /Users/OGN/1Projects/ZBrain2/app/assets/imagesbk


In [18]:
# get stack names

stacks = os.listdir(stack_folder)
stacks = [ stack for stack in stacks if os.path.isdir(stack) ]
print stacks[:5]

['6.7FRhcrtR-Gal4-uasKaede', 'Anti-5HT', 'Anti-Gad67', 'Anti-GlyR', 'Anti-tERK']


In [29]:
# create color channel folders

colors = ['red', 'blue', 'green', 'cyan']

for stack in stacks:
    folder_path = "/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/%s/" % stack
    try:
        os.mkdir(folder_path)
    except:
        pass
    for color in colors:
        channel_path = folder_path + color
        try:
            os.mkdir(channel_path)
        except:
            pass

In [63]:
# create color channel overlays for each stack

for stack in stacks[84:]:
    src = "/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/%s/" % stack
    base = "/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/%s/" % stack

    os.chdir(src)
    print base
    for color in colors:
        dest = base + color
        #call(["mogrify", "-path", dest, "+level-colors", "black,%s"%color, "-fuzz", "9%", "-transparent", "black", "*.png"]
        cwd = os.getcwd()
        print cwd
        try:
            subprocess.check_output(["mogrify", "-path", dest, "-format", "png", "+level-colors", "black,%s"%color, "-fuzz", "9%", "-transparent", "black", "*.jpg"], stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            print err.output


/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/ZBB_y312-Gal4/
/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/ZBB_y312-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/ZBB_y312-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/ZBB_y312-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/ZBB_y312-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/ZBB_y313-Gal4/
/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/ZBB_y313-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/ZBB_y313-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/ZBB_y313-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/ZBB_y313-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/ZBB_y314-Gal4/
/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/ZBB_y314-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/ZBB_y314-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/ZBB_y314-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/ZBB_y314-Gal4
/Users/OGN/1Projects/ZBrain2/app/asse

In [83]:
# prepare postgreSQL table

create_table_command = (
"""
    DROP TABLE colorchannels;
    CREATE TABLE colorChannels (
        channel_id SERIAL PRIMARY KEY,
        channel_color TEXT NOT NULL,
        channel_name TEXT NOT NULL,
        channel_image_path TEXT NOT NULL
    );
""")

In [84]:
# connect to zbrain2db postgreSQL database

conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [85]:
# create table colorChannels

try:
    cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print error
finally:
    if conn is not None:
        conn.close()

In [74]:
from natsort import natsorted

In [88]:
# create array of table insert commands for each color channel image

channel_id = 0
insert_commands = []
for stack in stacks[:5]:
    src = "/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/%s/" % stack
    base = "/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/%s/" % stack
    
    colors = ['red', 'blue', 'green', 'cyan']
    for color in colors:
        channel_folder = base + color
        channels = os.listdir(channel_folder)
        channels = [ "%s/%s" % (channel_folder, channel) for channel in channels]
        channels = natsorted(channels)
        for channel_image_path in channels:
            insert_command = ("INSERT INTO colorChannels (channel_id, channel_color, channel_name, channel_image_path) values (%i, '%s', '%s', '%s')" 
                                 % (channel_id, color, stack, channel_image_path))
            channel_id += 1
            insert_commands.append(insert_command)


["INSERT INTO colorChannels (channel_id, channel_color, channel_name, channel_image_path) values (200, 'blue', '6.7FRhcrtR-Gal4-uasKaede', '/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/6.7FRhcrtR-Gal4-uasKaede/blue/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-62.png')", "INSERT INTO colorChannels (channel_id, channel_color, channel_name, channel_image_path) values (201, 'blue', '6.7FRhcrtR-Gal4-uasKaede', '/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/6.7FRhcrtR-Gal4-uasKaede/blue/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-63.png')", "INSERT INTO colorChannels (channel_id, channel_color, channel_name, channel_image_path) values (202, 'blue', '6.7FRhcrtR-Gal4-uasKaede', '/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/6.7FRhcrtR-Gal4-uasKaede/blue/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-64.png')", "INSERT INTO colorChannels (channel_id, channel_color, channel_name, channel_image_path) values (203, 'blue', '6.7FRhcrtR-Gal4-uasKaede', '/Users/OGN/1Pr

In [89]:
# use insert command array to create table entries for each color channel image

conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

for insert_command in insert_commands:
    try:
        cursor.execute(insert_command)
    except Exception, e:
        print repr(e)
        
cursor.close()
conn.commit()

In [92]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute('SELECT * FROM colorChannels ORDER BY channel_id')

rows = cursor.fetchall()
for row in rows[300:500]:
    print row

(300, 'green', '6.7FRhcrtR-Gal4-uasKaede', '/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/6.7FRhcrtR-Gal4-uasKaede/green/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-24.png')
(301, 'green', '6.7FRhcrtR-Gal4-uasKaede', '/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/6.7FRhcrtR-Gal4-uasKaede/green/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-25.png')
(302, 'green', '6.7FRhcrtR-Gal4-uasKaede', '/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/6.7FRhcrtR-Gal4-uasKaede/green/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-26.png')
(303, 'green', '6.7FRhcrtR-Gal4-uasKaede', '/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/6.7FRhcrtR-Gal4-uasKaede/green/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-27.png')
(304, 'green', '6.7FRhcrtR-Gal4-uasKaede', '/Users/OGN/1Projects/ZBrain2/app/assets/colorChannels/6.7FRhcrtR-Gal4-uasKaede/green/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-28.png')
(305, 'green', '6.7FRhcrtR-Gal4-uasKaede', '/Users/OGN/1Projects/ZBrai